# 05f - Vertex AI > Training > Hyperparameter Tuning Jobs With Tensorboard HPARAMS

**Training Jobs Overview:**  
Where a model gets trained is where it consumes computing resources.  With Vertex AI, you have choices for configuring the computing resources available at training.  This notebook is an example of an execution environment.  When it was set up there were choices for machine type and accelerators (GPUs).  

In the 05 notebook, the model training happened directly in the notebook.  The models were then imported to Vertex AI and deployed to an endpoint for online predictions. 

In this 05a-05f series of demonstrations, the same model is trained using managed computing resources in Vertex AI as custom training jobs.  These jobs will be demonstrated as:

-  Custom Job from a python file (05a) and python source distribution (05b)
-  Training Pipeline that trains and saves models from a python file (05c) and python source distribution (05d)
-  Hyperparameter Tuning Jobs from a python source distribution (05e and 05f)

**This Notebook: An extension of 05e**  
This notebook trains the same Tensorflow Keras model from 05 by first modifying and saving the training code to a python script.  Then a Python source distribution is built containing the script.  While this example fits nicely in a single script, larger examples will benefit from the flexibility offered by source distributions and this job gives an example of making the shift.  

The source distribution is then used as an input for a Vertex AI Training Custom Job that is also assigned compute resources and a container (pre-built) for executing the training in a managed service.  

The Custom Job is then used as the input for a Vertex AI Training Hyperparameter Tuning Job.  This runs and manages the tuning loops for the number of trials in each loop, collects the metric(s) and manages the parameters with the search algorithm for parameter modification. 

The training can be reviewed with Vertex AI's managed Tensorboard under Experiments > Experiments, or by clicking on the `05f...` job under Training > Hyperparameter Tuning Jobs and then clicking the 'Open Tensorboard' link.  **Click on the HParams tab in Tensorboard to review the hyperparameters and metrics.**

### Prerequisites:
-  01 - BigQuery - Table Data Source
-  Understanding:
    -  05 - Vertex AI > Notebooks - Models Built in Notebooks with Tensorflow
        -  Contains a more granular review of the Tensorflow model training

### Overview:
-  Setup
-  Connect to Tensorboard instance from 05
-  Create a `train.py` Python script that recreates the local training in 04a
-  Build a Python source distribution that contains the `train.py` script
-  Use Python Client google.cloud.aiplatform for Vertex AI
   -  Custom training job with aiplatform.CustomJob.from_local_script
   -  Hyperparameter tuning job with aiplatform.HyperparameterTuningJob
      -  Run job with .run
   -  Upload best Model to Vertex AI with aiplatform.Model.upload
   -  Create Endpoint with Vertex AI with aiplatform.Endpoint.create
      -  Deploy model to endpoint with .deploy 
-  Online Prediction demonstrated using Vertex AI Endpoint with deployed model
   -  Get records to score from BigQuery table
   -  Prediction with aiplatform.Endpoint.predict
   -  Prediction with REST
   -  Prediction with gcloud (CLI)

### Resources:
-  [BigQuery Tensorflow Reader](https://www.tensorflow.org/io/tutorials/bigquery)
-  [Keras Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)
   -  [Keras API](https://www.tensorflow.org/api_docs/python/tf/keras)
-  [Python Client For Google BigQuery](https://googleapis.dev/python/bigquery/latest/index.html)
-  [Tensorflow Python Client](https://www.tensorflow.org/api_docs/python/tf)
-  [Tensorflow I/O Python Client](https://www.tensorflow.org/io/api_docs/python/tfio/bigquery)
-  [Python Client for Vertex AI](https://googleapis.dev/python/aiplatform/latest/aiplatform.html)
-  [Create a Python source distribution](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container) for a Vertex AI custom training job
-  Containers for training (Pre-Built)
   -  [Overview](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)
   -  [List](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)
-  Vertex AI Hyperparameter Tuning
   -  [Overview of Hyperparameter Tuning](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)
   -  [Using Hyperparameter Tuning](https://cloud.google.com/vertex-ai/docs/training/using-hyperparameter-tuning)
-  [Tensorboard HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams)


---
## Vertex AI - Conceptual Flow

<img src="architectures/slides/05f_arch.png">

---
## Vertex AI - Workflow

<img src="architectures/slides/05f_console.png">

---
## Setup

inputs:

In [37]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'fraud'
NOTEBOOK = '05f'

# Resources
TRAIN_IMAGE = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-7:latest'
DEPLOY_IMAGE ='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest'
TRAIN_COMPUTE = 'n1-standard-4'
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [2]:
from google.cloud import aiplatform
from datetime import datetime

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bigquery = bigquery.Client()

parameters:

In [4]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [5]:
# Give service account roles/storage.objectAdmin permissions
# Console > IMA > Select Account <projectnumber>-compute@developer.gserviceaccount.com > edit - give role
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'764015827198-compute@developer.gserviceaccount.com'

environment:

In [6]:
!rm -rf {DIR}
!mkdir -p {DIR}

---
## Get Vertex AI Experiments Tensorboard Instance Name
[Vertex AI Experiments](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) has managed [Tensorboard](https://www.tensorflow.org/tensorboard) instances that you can track Tensorboard Experiments (a training run or hyperparameter tuning sweep).  

The training job will show up as an experiment for the Tensorboard instance and have the same name as the training job ID.

This code checks to see if a Tensorboard Instance has been created in the project, retrieves it if so, creates it otherwise:

In [14]:
tb = aiplatform.Tensorboard.list(filter=f'display_name={DATANAME}')
if tb:
    tb = tb[0]
else:
    tb = aiplatform.Tensorboard.create(display_name = DATANAME, labels = {'notebook':f'{DATANAME}'})

In [16]:
tb.resource_name

'projects/715288179162/locations/us-central1/tensorboards/6212451803166932992'

---
## Training

### Assemble Python File for Training

Create the main python trainer file as `/train.py`:

In [9]:
!mkdir -p {DIR}/source/trainer

In [10]:
%%writefile {DIR}/source/trainer/train.py

# package import
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from google.cloud import bigquery
import argparse
import os
import sys
import hypertune

# import argument to local variables
parser = argparse.ArgumentParser()
# the passed param, dest: a name for the param, default: if absent fetch this param from the OS, type: type to convert to, help: description of argument
parser.add_argument('--epochs', dest = 'epochs', default = 10, type = int, help = 'Number of Epochs')
parser.add_argument('--batch_size', dest = 'batch_size', default = 32, type = int, help = 'Batch Size')
parser.add_argument('--var_target', dest = 'var_target', type=str)
parser.add_argument('--var_omit', dest = 'var_omit', type=str, nargs='*')
parser.add_argument('--project_id', dest = 'project_id', type=str)
parser.add_argument('--dataname', dest = 'dataname', type=str)
parser.add_argument('--region', dest = 'region', type=str)
parser.add_argument('--notebook', dest = 'notebook', type=str)
# hyperparameters
parser.add_argument('--lr',dest='learning_rate', required=True, type=float, help='Learning Rate')
parser.add_argument('--m',dest='momentum', required=True, type=float, help='Momentum')
args = parser.parse_args()

# setup tensorboard hparams
#    "lr": aiplatform.hyperparameter_tuning.DoubleParameterSpec(min=0.001, max=0.1, scale="log"),
#    "m": aiplatform.hyperparameter_tuning.DoubleParameterSpec(min=1e-7, max=0.9, scale="linear")
HP_LEARNING_RATE = hp.HParam('learning_rate',hp.RealInterval(0.0, 1.0))
HP_MOMENTUM = hp.HParam('momentum', hp.RealInterval(0.0,1.0))
hparams = {
    HP_LEARNING_RATE: args.learning_rate,
    HP_MOMENTUM: args.momentum
}

# built in parameters for data source:
PROJECT_ID = args.project_id
DATANAME = args.dataname
REGION = args.region
NOTEBOOK = args.notebook

# clients
bigquery = bigquery.Client(project = PROJECT_ID)

# get schema from bigquery source
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{DATANAME}_prepped'"
schema = bigquery.query(query).to_dataframe()

# get number of classes from bigquery source
nclasses = bigquery.query(query = f'SELECT DISTINCT {args.var_target} FROM {DATANAME}.{DATANAME}_prepped WHERE {args.var_target} is not null').to_dataframe()
nclasses = nclasses.shape[0]

# prepare inputs for tensorflow training
OMIT = args.var_omit + ['splits']

selected_fields = schema[~schema.column_name.isin(OMIT)].column_name.tolist()

feature_columns = []
feature_layer_inputs = {}
for header in selected_fields:
    if header != args.var_target:
        feature_columns.append(tf.feature_column.numeric_column(header))
        feature_layer_inputs[header] = tf.keras.Input(shape=(1,),name=header)

# all the columns in this data source are either float64 or int64
output_types = schema[~schema.column_name.isin(OMIT)].data_type.tolist()
output_types = [dtypes.float64 if x=='FLOAT64' else dtypes.int64 for x in output_types]

# remap input data to Tensorflow inputs of features and target
def transTable(row_dict):
    target=row_dict.pop(args.var_target)
    target = tf.one_hot(tf.cast(target,tf.int64), nclasses)
    target = tf.cast(target, tf.float32)
    return(row_dict, target)

# function to setup a bigquery reader with Tensorflow I/O
def bq_reader(split):
    reader = BigQueryClient()

    training = reader.read_session(
        parent = f"projects/{PROJECT_ID}",
        project_id = PROJECT_ID,
        table_id = f"{DATANAME}_prepped",
        dataset_id = DATANAME,
        selected_fields = selected_fields,
        output_types = output_types,
        row_restriction = f"splits='{split}'",
        requested_streams = 3
    )
    
    return training

train = bq_reader('TRAIN').parallel_read_rows().map(transTable).shuffle(args.batch_size*3).batch(args.batch_size)
validate = bq_reader('VALIDATE').parallel_read_rows().map(transTable).batch(args.batch_size)
test = bq_reader('TEST').parallel_read_rows().map(transTable).batch(args.batch_size)

# define model and compile
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
feature_layer_outputs = feature_layer(feature_layer_inputs)

layers = tf.keras.layers.BatchNormalization()(feature_layer_outputs)
layers = tf.keras.layers.Dense(nclasses, activation = tf.nn.softmax)(layers)

model = tf.keras.Model(
    inputs = [v for v in feature_layer_inputs.values()],
    outputs = layers
)
opt = tf.keras.optimizers.SGD(learning_rate = hparams[HP_LEARNING_RATE], momentum = hparams[HP_MOMENTUM])
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer = opt,
    loss = loss,
    metrics = ['accuracy', tf.keras.metrics.AUC(curve='PR')]
)


# setup tensorboard logs and train
log_dir=os.environ['AIP_TENSORBOARD_LOG_DIR']
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1)
hparams_callback = hp.KerasCallback(log_dir + 'train/', hparams)
history = model.fit(train, epochs = args.epochs, callbacks = [tensorboard_callback, hparams_callback], validation_data = validate)

# output the model save files
model.save(os.getenv("AIP_MODEL_DIR"))

# report hypertune info back to Vertex AI Training > Hyperparamter Tuning Job
hpt = hypertune.HyperTune()
hpt.report_hyperparameter_tuning_metric(
    hyperparameter_metric_tag = 'loss',
    metric_value = history.history['loss'][-1],
    global_step = 1)

Writing temp/05f/source/trainer/train.py


### Assemble Python Source Distribution

create `setup.py` file:

In [11]:
%%writefile {DIR}/source/setup.py
from setuptools import setup
from setuptools import find_packages

REQUIRED_PACKAGES = ['tensorflow_io']

setup(
    name = 'trainer',
    version = '0.1',
    install_requires = REQUIRED_PACKAGES, 
    packages = find_packages(),
    include_package_data = True,
    description='Training Package'
)

Writing temp/05f/source/setup.py


add `__init__.py` file to the trainer modules folder:

In [12]:
!touch {DIR}/source/trainer/__init__.py

Create the source distribution and copy it to the projects storage bucket:
- change to the local direcory with the source folder
- remove any previous distributions
- tar and gzip the source folder
- copy the distribution to the project folder on GCS
- change back to the local project directory

In [13]:
%cd {DIR}

!rm -f source.tar source.tar.gz
!tar cvf source.tar source
!gzip source.tar
!gsutil cp source.tar.gz {URI}/{TIMESTAMP}/source.tar.gz

temp = '../'*(DIR.count('/')+1)
%cd {temp}

/home/jupyter/vertex-ai-mlops/temp/05f
source/
source/setup.py
source/trainer/
source/trainer/__init__.py
source/trainer/train.py
Copying file://source.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  2.4 KiB/  2.4 KiB]                                                
Operation completed over 1 objects/2.4 KiB.                                      
/home/jupyter/vertex-ai-mlops


### Setup Training Job

In [14]:
CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--batch_size=" + str(BATCH_SIZE),
    "--var_target=" + VAR_TARGET,
    "--var_omit=" + VAR_OMIT,
    "--project_id=" + PROJECT_ID,
    "--dataname=" + DATANAME,
    "--region=" + REGION,
    "--notebook=" + NOTEBOOK
]

In [15]:
MACHINE_SPEC = {
    "machine_type": TRAIN_COMPUTE,
    "accelerator_count": 0
}

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": MACHINE_SPEC,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [f"{URI}/{TIMESTAMP}/source.tar.gz"],
            "python_module": "trainer.train",
            "args": CMDARGS
        }
    }
]

In [16]:
customJob = aiplatform.CustomJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    worker_pool_specs = WORKER_POOL_SPEC,
    base_output_dir = f"{URI}/{TIMESTAMP}",
    staging_bucket = f"{URI}/{TIMESTAMP}",
    labels = {'notebook':f'{NOTEBOOK}'}
)

### Setup Hyperparameter Tuning Job

In [17]:
METRIC_SPEC = {
    "loss": "minimize"
}

PARAMETER_SPEC = {
    "lr": aiplatform.hyperparameter_tuning.DoubleParameterSpec(min=0.001, max=0.1, scale="log"),
    "m": aiplatform.hyperparameter_tuning.DoubleParameterSpec(min=1e-7, max=0.9, scale="linear")
}

In [18]:
htJob = aiplatform.HyperparameterTuningJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    custom_job = customJob,
    metric_spec = METRIC_SPEC,
    parameter_spec = PARAMETER_SPEC,
    max_trial_count = 20,
    parallel_trial_count = 5,
    search_algorithm = None,
    labels = {'notebook':f'{NOTEBOOK}'}
)

### Run Training Job

In [19]:
htJob.run(
    service_account = SERVICE_ACCOUNT,
    tensorboard = tb.resource_name
)

INFO:google.cloud.aiplatform.jobs:Creating HyperparameterTuningJob
INFO:google.cloud.aiplatform.jobs:HyperparameterTuningJob created. Resource name: projects/764015827198/locations/us-central1/hyperparameterTuningJobs/9023561087273926656
INFO:google.cloud.aiplatform.jobs:To use this HyperparameterTuningJob in another session:
INFO:google.cloud.aiplatform.jobs:hpt_job = aiplatform.HyperparameterTuningJob.get('projects/764015827198/locations/us-central1/hyperparameterTuningJobs/9023561087273926656')
INFO:google.cloud.aiplatform.jobs:View HyperparameterTuningJob:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/9023561087273926656?project=764015827198
INFO:google.cloud.aiplatform.jobs:View Tensorboard:
https://us-central1.tensorboard.googleusercontent.com/experiment/projects+764015827198+locations+us-central1+tensorboards+8225226585066700800+experiments+9023561087273926656
INFO:google.cloud.aiplatform.jobs:HyperparameterTuningJob projects/764015827198/locations/

### Review the Job in Tensorboard > HParams interface:

<img src="architectures/notebooks/05f_Screenshots/hparams.png">

In [20]:
# if trial.state.name == 'SUCCEEDED'
losses = [trial.final_measurement.metrics[0].value if trial.state.name == 'SUCCEEDED' else 1 for trial in htJob.trials]
losses

[0.0037689567543566227,
 0.0050681889988482,
 0.003995178267359734,
 0.005229730624705553,
 0.003925533965229988,
 0.011400873772799969,
 0.0039536431431770325,
 0.0038270000368356705,
 0.003537785494700074,
 0.0041256132535636425,
 0.003912755288183689,
 0.0034100953489542007,
 0.003473690478131175,
 0.0035315377172082663,
 0.003787487978115678,
 0.003514513373374939,
 0.003420517547056079,
 0.003489802358672023,
 0.003581529250368476,
 0.0035107736475765705]

In [21]:
best = htJob.trials[losses.index(min(losses))]
best

id: "12"
state: SUCCEEDED
parameters {
  parameter_id: "lr"
  value {
    number_value: 0.1
  }
}
parameters {
  parameter_id: "m"
  value {
    number_value: 0.41301483818051765
  }
}
final_measurement {
  step_count: 1
  metrics {
    metric_id: "loss"
    value: 0.0034100953489542007
  }
}
start_time {
  seconds: 1638534977
  nanos: 29666506
}
end_time {
  seconds: 1638535630
}

---
## Serving

### Upload The Model

In [22]:
model = aiplatform.Model.upload(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    serving_container_image_uri = DEPLOY_IMAGE,
    artifact_uri = f"{URI}/{TIMESTAMP}/{best.id}/model",
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/764015827198/locations/us-central1/models/4415994938915815424/operations/5825567814932496384
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/764015827198/locations/us-central1/models/4415994938915815424
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/764015827198/locations/us-central1/models/4415994938915815424')


In [23]:
model.display_name

'05f_fraud_20211203120602'

### Create An Endpoint

In [24]:
endpoint = aiplatform.Endpoint.create(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    labels = {'notebook':f'{NOTEBOOK}'}
)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/764015827198/locations/us-central1/endpoints/1165174462186782720/operations/721863537214881792
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/764015827198/locations/us-central1/endpoints/1165174462186782720
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/764015827198/locations/us-central1/endpoints/1165174462186782720')


In [25]:
endpoint.display_name

'05f_fraud_20211203120602'

### Deploy Model To Endpoint

In [26]:
endpoint.deploy(
    model = model,
    deployed_model_display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    traffic_percentage = 100,
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 1
)

INFO:google.cloud.aiplatform.models:Deploying Model projects/764015827198/locations/us-central1/models/4415994938915815424 to Endpoint : projects/764015827198/locations/us-central1/endpoints/1165174462186782720
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/764015827198/locations/us-central1/endpoints/1165174462186782720/operations/7089953410316763136
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/764015827198/locations/us-central1/endpoints/1165174462186782720


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [27]:
pred = bigquery.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 10").to_dataframe()

In [28]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,79470,1.073063,-0.026793,0.796502,1.754389,-0.219910,0.930349,-0.515894,0.461495,-0.132966,...,0.048539,-0.258218,0.171861,1.028847,-0.037692,-0.013125,0.0,0,07fdced0-3837-47a1-9526-64d74ad9b113,TEST
1,122082,1.878563,0.020785,-1.621113,2.908813,2.507847,4.709442,-0.830626,1.136154,-0.395755,...,0.121098,0.707538,0.140100,0.155684,0.016375,-0.053892,0.0,0,7c1f61ba-7586-414e-ba8a-1c4385d59933,TEST
2,11494,-1.169744,0.462878,1.587579,-1.252560,1.012817,-0.918413,0.817632,-0.522804,1.388247,...,-0.042144,-0.021269,-0.283979,0.556661,-0.191722,-0.071773,0.0,0,a3046c87-cee5-40fd-9302-4d230b823246,TEST
3,58900,-2.728403,-2.340346,2.551039,2.623092,3.014728,-0.807294,-2.286621,0.757561,-1.279297,...,0.370139,-0.212784,-0.104956,1.355781,-0.053063,0.125386,0.0,0,bc3f2800-a4bb-4077-b017-f55f03c4f00c,TEST


In [29]:
newob = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')[0]
#newob

In [30]:
instances = [json_format.ParseDict(newob, Value())]
parameters = json_format.ParseDict({}, Value())

### Get Predictions: Python Client

In [31]:
prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[0.999538898, 0.000461080112]], deployed_model_id='2291109553401495552', explanations=None)

In [32]:
prediction.predictions[0]

[0.999538898, 0.000461080112]

In [33]:
np.argmax(prediction.predictions[0])

0

### Get Predictions: REST

In [34]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [35]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    [
      0.999538898,
      0.000461080112
    ]
  ],
  "deployedModelId": "2291109553401495552",
  "model": "projects/764015827198/locations/us-central1/models/4415994938915815424",
  "modelDisplayName": "05f_fraud_20211203120602"
}


### Get Predictions: gcloud (CLI)

In [36]:
!gcloud beta ai endpoints predict {endpoint.name.rsplit('/',1)[-1]} --region={REGION} --json-request={DIR}/request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999538898, 0.000461080112]]


---
## Remove Resources
see notebook "99 - Cleanup"